[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/vector-search-lab/)

# Step 1: Setup prerequisites

Replace `<MONGODB_URI>` with your **MongoDB connection string**

In [25]:
import os
from pymongo import MongoClient

In [ ]:
# Retain the quotes ("") when pasting the URI
MONGODB_URI = "<MONGODB_URI>"
# Initialize a MongoDB Python client
mongodb_client = MongoClient(MONGODB_URI, appname="devrel.workshop.vector_search")
# Check the connection to the server
mongodb_client.admin.command("ping")

### **Do not change the values assigned to the variables below**

In [27]:
# Database name
DB_NAME = "mongodb_genai_devday"
# Collection name
COLLECTION_NAME = "books"
# Name of the vector search index
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

📚 https://pymongo.readthedocs.io/en/stable/tutorial.html#getting-a-database

In [ ]:
# Connect to the `DB_NAME` database.
# Use the `mongodb_client` instantiated above.
db = <CODE_BLOCK_1>

📚 https://pymongo.readthedocs.io/en/stable/tutorial.html#getting-a-collection

In [ ]:
# Connect to the `COLLECTION_NAME` collection.
# Use the `mongodb_client`, database and collection variables defined above.
collection = <CODE_BLOCK_2>

In [29]:
SERVERLESS_URL = os.environ.get("SERVERLESS_URL")

# Step 2: Import data into MongoDB

In [30]:
import requests

In [ ]:
response = requests.post(
    url=SERVERLESS_URL, json={"task": "import_vs_data", "data": MONGODB_URI}
)
if response.status_code == 200:
    print(
        f"{collection.count_documents({})} documents ingested into the books collection."
    )
else:
    raise Exception(response.json())

# Step 3: Generating embeddings

In [ ]:
# You may see a warning upon running this cell. You can ignore it.
from sentence_transformers import SentenceTransformer
from PIL import Image

In [58]:
# Load a multimodal embedding model using the Sentence Transformers library
embedding_model = SentenceTransformer("clip-ViT-B-32")

### For images

📚 https://huggingface.co/sentence-transformers/clip-ViT-B-32#usage

In [ ]:
image_url = "https://images.isbndb.com/covers/77/44/9780789467744.jpg"
# Load the image from the URL above
image = Image.open(requests.get(image_url, stream=True).raw)
# Embed the `image` using the `embedding_model` instantiated above and return the embedding as a list
# An array can be converted to a list using the `tolist()` method
embedding = <CODE_BLOCK_3>

In [ ]:
print(embedding)

### For text

In [36]:
text = "Puppy Preschool: Raising Your Puppy Right---Right from the Start!"
# Use the same `embedding_model` to embed a piece of text
embedding = embedding_model.encode(text).tolist()

In [ ]:
print(embedding)

# Step 4: Adding embeddings to existing data in Atlas

In [89]:
from typing import List, Dict, Optional

In [38]:
# Field in the documents to embed-- in this case, the book cover
field_to_embed = "cover"
# Name of the embedding field to add to the documents
embedding_field = "embedding"

In [59]:
def get_embedding(content: str, mode: str) -> List[float]:
    """
    Generate embeddings

    Args:
        content (str): Content to embed
        mode (str): Content mode (Can be one of "image" or "text")

    Returns:
        List[float]: Embedding of the content as a list.
    """
    # If an image URL is provided, first load the image
    if mode == "image":
        content = Image.open(requests.get(content, stream=True).raw)
    return embedding_model.encode(content).tolist()

📚 https://www.mongodb.com/docs/manual/tutorial/query-documents/#select-all-documents-in-a-collection

In [ ]:
# Query for all documents in the `collection` collection.
results = <CODE_BLOCK_4>

📚 **$set:** https://www.mongodb.com/docs/manual/reference/operator/update/set/#syntax

📚 **update_one():** https://www.mongodb.com/docs/manual/reference/method/db.collection.updateOne/#definition

In [ ]:
# Update each document in the `collection` collection with embeddings
for result in results:
    content = result[field_to_embed]
    # Use the `get_embedding` function defined above to embed the `content`
    # Note that `content` contains the cover image URL for the book 
    embedding = <CODE_BLOCK_5>

    
    # Filter for the document where the `_id` field is equal to the `_id` of the current document
    filter = {"_id": result["_id"]}
    # Set the `embedding_field` field to the value `embedding` using the `$set` operator
    update = <CODE_BLOCK_6>
    # Update the documents in the `collection` collection inplace using the `update_one()` operation
    # Get the right document `_id` using the `filter` and apply the `update`
    <CODE_BLOCK_7>

# Step 5: Create a vector search index

In [46]:
# Create vector index definition specifying:
# path: Path to the embeddings field
# numDimensions: Number of embedding dimensions- depends on the embedding model used
# similarity: Similarity metric. One of cosine, euclidean, dotProduct.
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "cosine",
            }
        ]
    },
}

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_search_index

In [ ]:
# Create a vector search index with the above definition for the `collection` collection
<CODE_BLOCK_8>

# Step 6: Perform vector search queries

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Filter Example")

In [ ]:
# Define a function to retrieve relevant documents for a user query using vector search
def vector_search(
    user_query: str, mode: str, filter: Optional[Dict] = {}
) -> None:
    """
    Retrieve relevant documents for a user query using vector search.

    Args:
    user_query (str): The user's query (can be a piece of text or a link to an image)
    mode (str): Query mode (image or text)
    filter (Optional[Dict], optional): Optional vector search pre-filter
    """
    # Generate embedding for the `user_query` using the `get_embedding` function defined in Step 4
    query_embedding = <CODE_BLOCK_9>

    # Define an aggregation pipeline consisting of a $vectorSearch stage, followed by a $project stage
    # Set the number of candidates to 50 and only return the top 5 documents from the vector search
    # Set the `filter` field in the $vectorSearch stage to the value `filter` passed to the function
    # In the $project stage, exclude the `_id` field, include these fields: `title`, `cover`, `year`, `pages`, and the `vectorSearchScore`
    # NOTE: Use variables defined previously for the `index`, `queryVector` and `path` fields in the $vectorSearch stage
    pipeline = <CODE_BLOCK_10>

    # Execute the aggregation `pipeline` and store the results in `results`
    results = <CODE_BLOCK_11>

    # Print book title, score, and cover image
    for book in results:
        cover = Image.open(requests.get(book.get("cover"), stream=True).raw).resize((100,150))
        print(f"{book.get('title')}({book.get('year')}, {book.get('pages')} pages): {book.get('score')}")
        display(cover)

In [ ]:
# Test the vector search with a text query
vector_search("A peaceful and uplifting atmosphere", "text")

# Also try these text queries:
# - Healthy living and fitness
# - A man wearing a golden crown
# - Adorable fluffy creatures
# - Winning business agreement
# - A Winter's Tale on a Spanish Isle

In [ ]:
# Test the vector search with an image query
vector_search("https://images.isbndb.com/covers/26/12/9780385412612.jpg", "image")

# Also try these image queries:
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/1.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/2.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/3.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/4.png
# - https://mongodb-developer.github.io/vector-search-lab/img/query-sample/5.png

# Step 7: Adding pre-filters to your vector search

### Filter for books that were published after the year `2002`

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#about-the-filter-type

In [ ]:
# Modify the vector search index `model` from Step 5 to include the `year` field as a `filter` field
model = <CODE_BLOCK_12>

In [ ]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Check that the index update is complete before proceeding. To do so, ensure that the status of the index shows "Ready" in the Atlas UI.

📚 https://www.mongodb.com/docs/manual/reference/operator/query/gte/#syntax

In [ ]:
# Create a filter definition to filter for books where the `year` field is greater than `2002` using the `$gte` operator
filter = <CODE_BLOCK_13>
# Pass the `filter` as an argument to the `vector_search` function.
# Notice how this filter is incorporated in the `pipeline` in the `vector_search` function.
vector_search("A peaceful and uplifting atmosphere", "text", filter)

### Filter for books that were published after the year `2002` and under `250` pages

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#about-the-filter-type

In [ ]:
# Modify the vector search index `model` from Step 5 to include `year` and `pages` as filter fields
model = <CODE_BLOCK_14>

In [ ]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Check that the index update is complete before proceeding. To do so, ensure that the status of the index shows "Ready" in the Atlas UI.

📚 https://www.mongodb.com/docs/manual/reference/operator/query/lte/#mongodb-query-op.-lte

In [ ]:
# Create a filter definition to filter for books where the `year` field is greater than or equal to `2002` and the `pages` field is less than or equal to 250
# Use the `$gte` and `$lte` operators
filter = <CODE_BLOCK_15>
# Pass the `filter` as an argument to the `vector_search` function.
# Notice how this filter is incorporated in the `pipeline` in the `vector_search` function.
vector_search("A peaceful and uplifting atmosphere", "text", filter)

# Step 8: Changing the similarity metric

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#syntax

In [ ]:
# Modify the vector search index `model` from Step 5 to change the similarity metric to `dotProduct`
model = <CODE_BLOCK_16>

In [ ]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Check that the index update is complete before proceeding. To do so, ensure that the status of the index shows "Ready" in the Atlas UI.

In [ ]:
# Perform a vector search
# Note any differences in the results due to the different similarity metric
vector_search("A peaceful and uplifting atmosphere", "text")

# 🦹‍♀️ Enable vector quantization

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#syntax

In [ ]:
# Modify the vector search index `model` from Step 5 to use `scalar` quantization
model = <CODE_BLOCK_17>

In [ ]:
# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

# 🦹‍♀️ Hybrid search

In [123]:
# Name of the full-text search index
ATLAS_FTS_INDEX_NAME = "fts_index"

In [ ]:
# Create full-text search index definition specifying the field mappings
model = {
    "name": ATLAS_FTS_INDEX_NAME,
    "type": "search",
    "definition": {
        "mappings": {"dynamic": False, "fields": {"title": {"type": "string"}}}
    },
}

In [ ]:
# Create a full-text search index with the above definition for the `collection` collection
collection.create_search_index(model=model)

In [ ]:
# Reset the vector search index to the original vector search index definition
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 512,
                "similarity": "cosine",
            }
        ]
    },
}

# Update the vector search index with the modified model
collection.update_search_index(
    name=ATLAS_VECTOR_SEARCH_INDEX_NAME, definition=model["definition"]
)

**NOTE:** Refer to our [documentation](https://www.mongodb.com/docs/atlas/atlas-vector-search/tutorials/reciprocal-rank-fusion/#about-the-query) for a detailed explanation of the hybrid search query below.

In [ ]:
def hybrid_search(
    user_query: str, vector_weight: float, full_text_weight: float
) -> None:
    """
    Retrieve relevant documents for a user query using hybrid search.

    Args:
        user_query (str): User query string
        vector_weight (float): Weight of vector search in the final search results
        full_text_weight (float): Weight of full-text search in the final search results
    """
    pipeline = [
        # Vector search stage
        {
            "$vectorSearch": {
                "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
                "path": "embedding",
                "queryVector": get_embedding(user_query, "text"),
                "numCandidates": 50,
                "limit": 10,
            }
        },
        # Group all documents returned by the vector search into a single array named `docs`
        {"$group": {"_id": None, "docs": {"$push": "$$ROOT"}}},
        # Unwinds the array of documents in the `docs` field and stores the position of the document in the `docs` array, in a field named `rank`
        {"$unwind": {"path": "$docs", "includeArrayIndex": "rank"}},
        # Adds a new field named `vs_score` that contains the reciprocal rank score for each document in the `docs` array
        {
            "$addFields": {
                "vs_score": {
                    "$multiply": [
                        vector_weight,
                        {"$divide": [1.0, {"$add": ["$rank", 60]}]},
                    ]
                }
            }
        },
        # Includes only the fields `_id`, `title`, and `cover` and the `vs_score` in the results
        {
            "$project": {
                "vs_score": 1,
                "_id": "$docs._id",
                "title": "$docs.title",
                "cover": "$docs.cover",
            }
        },
        # Union with a full-text search stage
        {
            "$unionWith": {
                "coll": COLLECTION_NAME,
                "pipeline": [
                    # Full-text search
                    {
                        "$search": {
                            "index": ATLAS_FTS_INDEX_NAME,
                            "text": {"query": user_query, "path": "title"},
                        }
                    },
                    # Limits the number of documents returned by the full-text search to 10
                    {"$limit": 10},
                    # Group all documents returned by the full-text search into a single array named `docs`
                    {"$group": {"_id": None, "docs": {"$push": "$$ROOT"}}},
                    # Unwinds the array of documents in the `docs` field and stores the position of the document in the `docs` array, in a field named `rank`
                    {"$unwind": {"path": "$docs", "includeArrayIndex": "rank"}},
                    # Adds a new field named `fts_score` that contains the reciprocal rank score for each document in the `docs` array
                    {
                        "$addFields": {
                            "fts_score": {
                                "$multiply": [
                                    full_text_weight,
                                    {"$divide": [1.0, {"$add": ["$rank", 60]}]},
                                ]
                            }
                        }
                    },
                    # Includes only the fields `_id`, `title`, and `cover` and the `fts_score` in the results
                    {
                        "$project": {
                            "fts_score": 1,
                            "_id": "$docs._id",
                            "title": "$docs.title",
                            "cover": "$docs.cover",
                        }
                    },
                ],
            }
        },
        # The following stages combine the results from vector and full-text search into a single ranked list
        # Groups the documents in the results from the preceding stages by `title`, `vs_score`, and `fts_score`
        {
            "$group": {
                "_id": "$_id",
                "title": {"$first": "$title"},
                "vs_score": {"$max": "$vs_score"},
                "fts_score": {"$max": "$fts_score"},
                "cover": {"$first": "$cover"},
            }
        },
        # Includes only the fields `_id`, `title`, `cover`, `vs_score` and `fts_score` in the results
        {
            "$project": {
                "_id": 1,
                "title": 1,
                "vs_score": {"$ifNull": ["$vs_score", 0]},
                "fts_score": {"$ifNull": ["$fts_score", 0]},
                "cover": 1,
            }
        },
        # Adds a field named `score` that contains the sum of `vs_score` and `fts_score` to the results
        {
            "$project": {
                "score": {"$add": ["$fts_score", "$vs_score"]},
                "_id": 1,
                "title": 1,
                "vs_score": 1,
                "fts_score": 1,
                "cover": 1,
            }
        },
        # Sorts the results by descending order of `score`
        {"$sort": {"score": -1}},
        # Limits the output to 5 results only
        {"$limit": 5},
    ]

    results = collection.aggregate(pipeline)

    # Print book title, scores, and cover image
    for book in results:
        cover = Image.open(requests.get(book["cover"], stream=True).raw).resize(
            (100, 150)
        )
        print(
            f"{book.get('title')}, VS Score: {book.get('vs_score')}, FTS Score: {book.get('fts_score')}"
        )
        display(cover)

In [ ]:
# Test the hybrid search query with a weight of 1.0 for vector search and 0.0 for full-text search
hybrid_search(
    user_query="A Winter's Tale on a Spanish Isle",
    vector_weight=1.0,
    full_text_weight=0.0,
)

In [ ]:
# Test the hybrid search query with a weight of 0.5 for vector search and 0.5 for full-text search
hybrid_search(
    user_query="A Winter's Tale on a Spanish Isle",
    vector_weight=0.5,
    full_text_weight=0.5,
)